In [1]:
import tisane as ts
import pandas as pd
import os

df = pd.read_csv("pigs.csv")

## Initialize variables with data
# Bind measures to units at the time of declaration
week = ts.SetUp("Time", cardinality=12)
pig = ts.Unit("Pig", cardinality=82)  # 82 pigs
litter = ts.Unit("Litter", cardinality=22)  # 22 litters
# Each pig has 1 instance of an ordinal Evit measure
vitamin_e = pig.ordinal(
    "Evit", order=["Evit000", "Evit100", "Evit200"], number_of_instances=1
)
# Each pig has 1 instance of an ordinal Cu measure
copper = pig.ordinal("Cu", order=["Cu000", "Cu035", "Cu175"], number_of_instances=1)
# Each pig has for each value of week 1 instance of a numeric Weight measure
# Also: Each pig has 1 instance of a Weight measure corresponding to each week
weight = pig.numeric("Weight", number_of_instances=week)
# Each pig has for each value of week 1 instance of a numeric Feed consumption measure
feed = pig.numeric("Feed consumption", number_of_instances=week)

## Conceptual relationships
week.causes(weight)

## Data measurement relationships
# Pigs are nested within litters
pig.nests_within(litter)

## Specify and execute query
design = ts.Design(dv=weight, ivs=[week]).assign_data(df)

ts.infer_statistical_model_from_design(design=design, jupyter=True)

adding relationship <class 'tisane.variable.Has'>
Adding
adding relationship <class 'tisane.variable.Causes'>
Adding
adding relationship <class 'tisane.variable.Causes'>
Adding
adding relationship <class 'tisane.variable.Nests'>
Adding
adding relationship <class 'tisane.variable.Has'>
Adding
adding relationship <class 'tisane.variable.Has'>
Adding
adding relationship <class 'tisane.variable.Has'>
Adding


read through {filename}
Writing out path


In [3]:

# Tisane inferred the following statistical model based on this query:  {}

import pandas as pd
from pymer4.models import Lmer # supports Generalized linear models with or without mixed effects
import rpy2.robjects.lib.ggplot2 as ggplot2 # for visualizing residual plots to diagnose model fit


def createModel():
    # Dataframe is stored in local file: data.csv
    # You may want to replace the data path with an existing data file you already have.
    # You may also set df equal to a pandas dataframe you are already working with. 
    df = pd.read_csv('pigs.csv') # Make sure that the data path is correct


    model = Lmer(formula='Weight ~ Time + (1|Litter) + (1|Pig) + (1|Time)', family="gaussian", data=df)
    print(model.fit())
    return model

if __name__ == "main":
    createModel()






Formula: Weight~Time+(1|Litter)+(1|Pig)+(1|Time)

Family: gaussian	 Inference: parametric

Number of observations: 861	 Groups: {'Pig': 72.0, 'Litter': 21.0, 'Time': 12.0}

Log-likelihood: -2355.822 	 AIC: 4711.644

Random effects:

                 Name     Var    Std
Pig       (Intercept)  31.315  5.596
Litter    (Intercept)   9.528  3.087
Time      (Intercept)   1.764  1.328
Residual                9.757  3.124

No random effect correlations specified

Fixed effects:

             Estimate  2.5_ci  97.5_ci     SE      DF  T-stat  P-val  Sig
(Intercept)    15.686  13.195   18.177  1.271  27.926  12.342    0.0  ***
Time            6.943   6.718    7.169  0.115  10.013  60.228    0.0  ***
